<div>
    <img src="./images/sunpy_logo.png" width="500" align="left"/>
    <br clear="left"/> <!-- This line ensures that the text appears below the image -->
</div>

# Example notebook and exercises!

Now using all the information we have learned, lets analyse the flare event that occured on 2022-04-02 13:00 that was observed by Solar Orbiter and Earth.

As part of this notebook, I'll show some example workflow analysis you could do - but as an exercise, I suggest that you go through our documentation on [sunpy.org](https://sunpy.org/) and our [affiliated packages](https://sunpy.org/affiliated/) (e.g. [sunkit-image](https://docs.sunpy.org/projects/sunkit-image/en/latest/), [sunkit-instrument](https://docs.sunpy.org/projects/sunkit-instruments/en/stable/), and through the [example gallery](https://docs.sunpy.org/en/stable/generated/gallery/index.html) to see what more fun analysis you could do!

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

from sunpy.net import Fido, attrs as a
from sunpy.time import parse_time
import sunpy.map
import sunpy.timeseries as ts
from sunpy.coordinates import frames, get_horizons_coord
from astropy.visualization import ImageNormalize
import sunpy_soar
from aiapy.calibrate import register, update_pointing
import os

import numpy as np
import glob
import matplotlib.pyplot as plt
from sunpy.physics.differential_rotation import solar_rotate_coordinate
from matplotlib import colors
import hvpy

Lets first create a search and download data of interest, for this, lets build a Fido query like before.

Lets first search for GOES XRS data

In [ ]:
xrs_res = Fido.search(a.Time("2022-04-02 12:50", "2022-04-02 15:00"), 
                      a.Instrument.xrs, a.goes.SatelliteNumber(16), a.Resolution.flx1s)

In [ ]:
xrs_res

In [ ]:
Fido.fetch(xrs_res, path="./{instrument}/{file}")

In [ ]:
goes_xrs = ts.TimeSeries("./XRS/sci_xrsf-l2-flx1s_g16_d20220402_v2-2-0.nc")

In [ ]:
fig, ax = plt.subplots()
goes_xrs.plot()
ax.axvline(parse_time("2022-04-02 12:50").datetime, color='k')
ax.axvline(parse_time("2022-04-02 15:00").datetime, color='k')


Lets now search for AIA 171 Å data and Solar Orbiter EUI/FSI 174 Å data between teh vertical lines in teh GOES/XRS plot over the flare of interest

In [ ]:
image_res = Fido.search(a.Time("2022-04-02 12:00", "2022-04-02 14:30"),
                       (a.Instrument.aia & a.Wavelength(171*u.angstrom) & a.Sample(10*u.min)) | 
                       (a.Instrument.eui & a.Level(2) & a.soar.Product("EUI-FSI174-IMAGE")))

In [ ]:
# Fido.fetch(image_res, path="./{instrument}/{file}")

In [ ]:
aia_files = glob.glob("./AIA/*171a*.fits"); aia_files.sort()
eui_files = glob.glob("./EUI/*fsi174*.fits"); eui_files.sort()

In [ ]:
aia_maps = sunpy.map.Map(aia_files)

In [ ]:
eui_maps = sunpy.map.Map(eui_files)

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax1 = fig.add_subplot(1, 2, 1, projection=aia_maps[0])
ax2 = fig.add_subplot(1, 2, 2, projection=eui_maps[0])

aia_maps[0].plot(axes=ax1)
eui_maps[0].draw_limb(color='r', axes=ax1)
eui_maps[0].plot(axes=ax2)
aia_maps[0].draw_limb(color='g', axes=ax2)


## Prep AIA data from Level 1 to Level 1.5

To do this we will use sunpy's affiliated package `aiapy`

In [ ]:
aia_maps[0].meta["lvl_num"]

In [ ]:
m_updated_pointing = update_pointing(aia_maps[0])
m_registered = register(m_updated_pointing)

In [ ]:
m_registered.meta["lvl_num"]

In [ ]:
m_registered.plot(clip_interval=[5, 99.98]*u.percent)
plt.colorbar()

## TO DO: Try write a function that takes the input of an AIA map and updates the pointing and registers it to a Level 1.5 map 

Try yourself to do this

## Lets zoom in and rotate EUI maps

In [ ]:
eui_maps[0].plot()

In [ ]:
def make_zoom_eui(m):
    bl = SkyCoord(-5000*u.arcsec, -5000*u.arcsec, frame=m.coordinate_frame)
    tr = SkyCoord(5000*u.arcsec, 5000*u.arcsec, frame=m.coordinate_frame)
    return m.submap(bl, top_right=tr).rotate(missing=m.data.min())

In [ ]:
make_zoom_eui(eui_maps[0]).plot()

In [ ]:
bl = SkyCoord(700*u.arcsec, 200*u.arcsec, frame=aia_maps[0].coordinate_frame)
tr = SkyCoord(850*u.arcsec, 400*u.arcsec, frame=aia_maps[0].coordinate_frame)


In [ ]:
fig = plt.figure(figsize=(11, 5))
ax1 = fig.add_subplot(1, 2, 1, projection=aia_maps[0])
ax2 = fig.add_subplot(1, 2, 2, projection=eui_maps[0])
aia_maps[0].plot(axes=ax1)
aia_maps[0].draw_quadrangle(bl, top_right=tr, axes=ax1, edgecolor='r')

eui_maps[0].plot(axes=ax2)
eui_maps[0].draw_quadrangle(bl, top_right=tr, axes=ax2, edgecolor='r')

## TO DO: crop all the EUI maps to this field of view

## Lets look at SOHO LASCO C2 data

For this example, lets demonstrate how you can use jp2 files from helioviewer. Using hvpy, we can download jp2 files. Here, we'll use sunpy to find times for which there is SOHO LASCO C2 data, and then we'll use hvpy to download the processed jp2 LASCO C2 files, and use sunpy to read and plot them.

In [ ]:
result = Fido.search(a.Time("2022-04-02 12:50", "2022-04-02 17:00"),
                     a.Instrument.lasco,
                     a.Detector.c2)

In [ ]:
if not os.path.exists("./LASCO_C2"):
    os.mkdir("LASCO_C2")

lasco_files = []
for i in range(len(result["VSO"])):
    time_map = parse_time(result["VSO"]["Start Time"].datetime).datetime[i]
    cor2_file = hvpy.save_file(hvpy.getJP2Image(time_map,
                                                hvpy.DataSource.LASCO_C2.value),
                               "./LASCO_C2/lasco_c2_{:s}.jp2".format(time_map.strftime("%Y%m%d%H%M%S")), 
                               overwrite=True)
    lasco_files.append(cor2_file)

In [ ]:
lasco_files = glob.glob("./LASCO_C2/*.jp2"); lasco_files.sort()
lasco_maps = sunpy.map.Map(lasco_files, sequence=True)

In [ ]:
lasco_maps[0].plot()
lasco_maps[0].draw_limb(color='w')

In [ ]:
fig = plt.figure()
ani = lasco_maps.plot(norm=ImageNormalize(vmin=0, vmax=250,
                                          stretch=lasco_maps[0].plot_settings['norm'].stretch))
ani.save('lasco-maps-seq.mp4', fps=15, dpi=300)

# Lets plot the AIA and LASCO C2 map together

One thing to be aware of, is that given that the times and teh positions of the spacecraft are different (even though small), they have a different WCS. Hence to plot them on the same plot, we need to reproject one map to another. Here lets reproject the aia map to the lasco map wcs. One final thing to note is that the `rsun_ref` should be the same for both maps. Please read this topic guide about this: https://docs.sunpy.org/en/stable/topic_guide/rsun.html

In [ ]:
aia_maps[6].meta["rsun_ref"] = sunpy.sun.constants.radius.to_value(u.m)
lasco_maps[0].meta["rsun_ref"] = sunpy.sun.constants.radius.to_value(u.m)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(projection=lasco_maps[6])
lasco_maps[6].plot(vmin=0, vmax=250)
with frames.Helioprojective.assume_spherical_screen(lasco_maps[6].observer_coordinate):
    aia_map_lasco_wcs = aia_maps[13].resample([250, 250]*u.pix).reproject_to(lasco_maps[6].wcs)
aia_map_lasco_wcs.plot()


## Find HEK areas on the Sun

Lets now show an example of querying the Helio Event Knowledgebase (HEK) to find active regions around the time of thh flare event. Lets first start with finding HMI magnetogram data, and then querying active region data

In [ ]:
res = Fido.search(a.Time("2022-04-02 12:00", "2022-04-02 13:00"), 
                  a.Instrument.hmi, a.Physobs.los_magnetic_field)

In [ ]:
hmi_file = Fido.fetch(res[0][0], path="./{instrument}/{file}")

In [ ]:
hmi_map = sunpy.map.Map(hmi_file)

In [ ]:
hmi_rotated = hmi_map.rotate(order=3)


In [ ]:
hmi_rotated.plot()

In [ ]:
hek_results = Fido.search(a.Time("2022-04-02 12:01", "2022-04-02 14:00"),
                              a.hek.AR, a.hek.FRM.Name=='HMI SHARP')

In [ ]:
hek_results["hek"]["event_starttime", "event_endtime"]

In [ ]:
ar_areas = []
for i in range(len(hek_results["hek"])):
    ch = hek_results["hek"][i]
    p1 = ch["hpc_bbox"][9:-2].split(',')
    coord_values = [v.split(" ") for v in p1]
    ch_date = parse_time(ch['event_starttime'])
    ch_boundary = SkyCoord(
        [(float(v[0]), float(v[1])) * u.arcsec for v in coord_values],
        obstime=ch_date, observer="earth",
        frame=frames.Helioprojective)
    #rotated_ch_boundary = solar_rotate_coordinate(ch_boundary, observer=hmi_map.observer_coordinate)
    ar_areas.append(ch_boundary)


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection=hmi_rotated)
hmi_rotated.plot(axes=ax, norm=colors.Normalize(vmin=-500, vmax=500))
hmi_rotated.draw_limb()
for ar in ar_areas:
    ax.plot_coord(ar)

plt.colorbar()
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection=aia_maps[0])
aia_maps[0].plot(axes=ax, clip_interval=[2, 99.98]*u.percent)
aia_maps[0].draw_limb()
for ar in ar_areas:
    ax.plot_coord(ar)
eui_maps[0].draw_limb()
plt.colorbar()
plt.show()

In [ ]:
eui_maps[0].plot()
ax = plt.gca()
for ar in ar_areas:
    transformed_coords = ar.transform_to(eui_maps[0].coordinate_frame)
    if np.any(transformed_coords.is_visible(tolerance=1*u.km)):
        ax.plot_coord(transformed_coords[transformed_coords.is_visible(tolerance=1*u.km)])
hmi_map.draw_limb()